In [1]:
# Import modules
import sys

In [2]:
# Import custom modules
sys.path.append('../')

from src.models.upload_data import *

In [3]:
# Set notebook attributes
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
# Set magic attributes
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
# Configuration
args = argparse.ArgumentParser()
args.add_argument("--config", default="../params.yaml")
parsed_args = vars(args.parse_args(""))
config_path = parsed_args['config']

### Defaulters with Limits

In [6]:
%%time
# Data ingestion, cleaning/wrangling
# convert_to_parquet(config_path, 'ftd', 'excel')
df_ftd_raw = pull_data(config_path, ftd_sql(), 'DWH', 'ftd', False, 'raw')

Currently loading ftd data set ...
Time taken is 0 seconds ...


,store_number,update_flag,reinstatement_reason
17,632642,1,first time defaulters during election risk mitigation measures
9,7068691,1,first time defaulters during election risk mitigation measures


---------------------------------------------------------------------------------------------------------------------------------------
CPU times: total: 31.2 ms
Wall time: 170 ms


In [7]:
%%time
# Data ingestion, cleaning/wrangling
df_ftd_clean = clean_dataset(config_path, 'ftd')


Currently cleaning ftd data set ...


,store_number,update_flag,reinstatement_reason
10,759602,1,first time defaulters during election risk mitigation measures


---------------------------------------------------------------------------------------------------------------------------------------
CPU times: total: 31.2 ms
Wall time: 85.1 ms


In [8]:
# Final dataset schema
df_ftd_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39 entries, 0 to 42
Data columns (total 3 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   store_number          39 non-null     object
 1   update_flag           39 non-null     int64 
 2   reinstatement_reason  39 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.2+ KB


In [9]:
df_defaulters_with_limits_raw = pd.read_excel('../data/raw/20.5_AH_Investigations_Limit_Allocation.xlsx', sheet_name=0)
df_defaulters_with_limits_raw.columns = df_defaulters_with_limits_raw.columns.str.lower().str.replace('[ /]', '_').str.replace('-_', '')
df_defaulters_with_limits_raw['store_number'] = df_defaulters_with_limits_raw['store_number'].astype('str')
df_defaulters_with_limits_raw.sample(2)

C:\Users\FM\AppData\Local\Temp\ipykernel_5108\4176765529.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df_defaulters_with_limits_raw.columns = df_defaulters_with_limits_raw.columns.str.lower().str.replace('[ /]', '_').str.replace('-_', '')


,store_number,loan_date,loan_term,days_past_due,safaricom_balance_as_at_13_dec_2022,max_limit_allocated_20221208
16,7823203,2022-07-22,7,137,14913.63,23000
6,7491730,2022-10-16,7,51,105847.38,75800


In [10]:
df_defaulters_with_limits_raw.shape

(25, 6)

In [11]:
df_defaulters_with_limits_raw['store_number'].nunique()

25

In [12]:
df_match = df_defaulters_with_limits_raw[(df_defaulters_with_limits_raw['store_number'].isin(df_ftd_clean['store_number'].unique()))]
df_match.shape

(24, 6)

In [13]:
df_not_match = df_defaulters_with_limits_raw[~(df_defaulters_with_limits_raw['store_number'].isin(df_ftd_clean['store_number'].unique()))]
df_not_match.shape

(1, 6)

In [14]:
df_not_match

,store_number,loan_date,loan_term,days_past_due,safaricom_balance_as_at_13_dec_2022,max_limit_allocated_20221208
6,7491730,2022-10-16,7,51,105847.38,75800
